# Лабораторная работа №3
## Выполнила студентка группы Зайцева Александра БВТ2004

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
import time as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict as dictionary

string = input('Введите строку: ')
substr = input('Введите подстроку: ')
ignore = bool(input('Игнорировать регистр(0/1): '))

print("\n"+str(string.find(substr)))
start_time = t.perf_counter()
string.find(substr)
print("Стандартное время поска подстроки: {:6f}".format((t.perf_counter() - start_time)*1000))


Введите строку: лилилась лилилось
Введите подстроку: лило
Игнорировать регистр(0/1): 1

11
Стандартное время поска подстроки: 0.287500


### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
def pref(str, j = 1, i = 0):
    p = [0]*len(str)
    while j < len(str):
        if str[i] == str[j]:
            p[j] = i + 1
            j, i = j + 1, i + 1
        elif i == 0:
            p[j] = 0
            j += 1
        else:
            i = p[i - 1]
    return p

def kmp(String:str, Substring:str, isignore = False) -> str:
    if isignore:
        String = String.lower()
        Substring = Substring.lower()
    p = pref(Substring)
    i, j = 0, 0
    while i < len(String):
        if String[i] == Substring[j]:
            i, j = i + 1, j + 1
            if j == len(Substring):
                return i-len(Substring)
        elif j > 0:
            j = p[j-1]
        else:
            i += 1
    return -1

print(kmp(string,substr,ignore))
start_time = t.perf_counter()
kmp(string,substr,ignore)
print("Время работы алгоритма: {:6f} ms".format((t.perf_counter() - start_time)*1000))

11
Время работы алгоритма: 0.281400 ms


#### Упрощенный алгоритм Бойера-Мура

In [3]:
def offset(Substring):
    for i in range(len(Substring)-2,-1,-1):
        if Substring[i] not in dict:
            dict[Substring[i]] = len(Substring)-i-1
    if Substring[len(Substring)-1] not in dict:
        dict[Substring[len(Substring)-1]] = len(Substring)
    dict['*'] = len(Substring)
def bm(String:str,Substring:str,isIgnore = False):
    offset(Substring)
    if isIgnore:
        String = String.lower()
        Substring = Substring.lower()
    i, j = len(Substring)-1, len(Substring)-1
    while i < len(String):
        if String[i] == Substring[j]:
            if j == 0:
                return i
            i -= 1
            j -= 1
        elif String[i] in dict:
            i += dict.get(String[i])
            j = len(Substring) - 1
        else:
            i += dict.get('*')
    return -1

dict = dictionary(int)
print(bm(string,substr))
start_time = t.perf_counter()
bm(string,substr,ignore)
print("Время работы алгоритма: {:6f} ms".format((t.perf_counter() - start_time)*1000))

11
Время работы алгоритма: 0.461400 ms


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [3]:
import math
from queue import PriorityQueue

N = 4 #размерность игры
SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

#проверка решаемости позиции
def reshenie(mass): 
    summ=int(0)
    for i in range(len(mass)):
        for j in range(i, len(mass)):
            if mass[i]==0:
                summ=summ+math.floor((i+1)/4)
            elif mass[i]>mass[j]:
                summ += 1
    return summ%2==1

def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # вниз
    if i < N - 1: offsets.append(N)  # вверх
    if j > 0: offsets.append(-1)     # вправо
    if j < N - 1: offsets.append(1)  # влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))

class Position:
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    def __lt__(self, other):
        return self.start_distance < other.start_distance

if reshenie(start) == 0:
    print('Нерешаемая позиция')
else:
    # картеж
    start = tuple(start)
    p = Position(start, 0)
    # очередь 
    candidates = PriorityQueue()
    candidates.put(p)
    # множество 
    visited = set([p])

    came_from = {p.position: None}
    
    while p.position != SOLVED:
        p = candidates.get()
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)
    path = []
    prev = p
    while p.position != start:
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

[15, 14, 10, 13, 9, 10, 14, 15]


### Вывод